<a href="https://colab.research.google.com/github/badrooxe/Team-Dynamics-Prediction-Using-the-16-Personality-Factors-16PF-/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('data.csv', sep='\t')
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826


In [3]:
question_map = {
    "A1": "I take time out for others.",
    "A2": "I know that I am not a special person.",
    "A3": "I take control of things.",
    "A4": "I try to forgive and forget.",
    "A5": "I keep in the background.",
    "A6": "I can't do without the company of others.",
    "A7": "I trust others.",
    "A8": "I am not easily frustrated.",
    "A9": "I cheer people up.",
    "A10": "I often feel uncomfortable around others.",

    "B1": "I seldom feel blue.",
    "B2": "I dislike myself.",
    "B3": "I take charge.",
    "B4": "I let others make the decisions.",
    "B5": "I believe in the importance of art.",
    "B6": "I like to get lost in thought.",
    "B7": "I wait for others to lead the way.",
    "B8": "I am willing to talk about myself.",
    "B9": "I find it difficult to approach others.",
    "B10": "I enjoy my privacy.",

    "C1": "I swim against the current.",
    "C2": "I feel guilty when I say 'no.'",
    "C3": "I am hard to get to know.",
    "C4": "I don't talk a lot.",
    "C5": "I believe in one true religion.",
    "C6": "I am not easily annoyed.",
    "C7": "I feel crushed by setbacks.",
    "C8": "I am afraid that I will do the wrong thing.",
    "C9": "I enjoy being part of a loud crowd.",
    "C10": "I weigh the pros against the cons.",

    "D1": "I do unexpected things.",
    "D2": "I get angry easily.",
    "D3": "I am quiet around strangers.",
    "D4": "I don't mind eating alone.",
    "D5": "I make people feel at ease.",
    "D6": "I use my brain.",
    "D7": "I have a good word for everyone.",
    "D8": "I feel desperate.",
    "D9": "I want to be in charge.",
    "D10": "I feel comfortable around people.",

    "E1": "I am the life of the party.",
    "E2": "I don't let others discourage me.",
    "E3": "I enjoy being part of a group.",
    "E4": "I love to daydream.",
    "E5": "I distrust people.",
    "E6": "I worry about things.",
    "E7": "I am not easily bothered by things.",
    "E8": "I respect authority.",
    "E9": "I do things that others find strange.",
    "E10": "I skip difficult words while reading.",

    "F1": "I feel comfortable with myself.",
    "F2": "I am exacting in my work.",
    "F3": "I tend to analyze things.",
    "F4": "I continue until everything is perfect.",
    "F5": "I believe that people are basically moral.",
    "F6": "I am quick to judge others.",
    "F7": "I am relaxed most of the time.",
    "F8": "I enjoy silence.",
    "F9": "I show my feelings.",
    "F10": "I judge people by their appearance.",

    "G1": "I prefer variety to routine.",
    "G2": "I never challenge things.",
    "G3": "I can't stand being contradicted.",
    "G4": "I try not to think about the needy.",
    "G5": "I am easily put out.",
    "G6": "I prefer to do things by myself.",
    "G7": "I get irritated easily.",
    "G8": "I know the answers to many questions.",
    "G9": "I trust what people say.",
    "G10": "I like to stand during the national anthem.",

    "H1": "I love flowers.",
    "H2": "I find it hard to forgive others.",
    "H3": "I leave my belongings around.",
    "H4": "I feel others' emotions.",
    "H5": "I let myself be pushed around.",
    "H6": "I don't like crowded events.",
    "H7": "I enjoy hearing new ideas.",
    "H8": "I act wild and crazy.",
    "H9": "I read a lot.",
    "H10": "I try to follow the rules.",

    "I1": "I enjoy wild flights of fantasy.",
    "I2": "I use swear words.",
    "I3": "I don't worry about things that have already happened.",
    "I4": "I say what I think.",
    "I5": "I am easily hurt.",
    "I6": "I enjoy spending time by myself.",
    "I7": "I don't mind being the center of attention.",
    "I8": "I seldom get lost in thought.",
    "I9": "I seldom daydream.",
    "I10": "I suspect hidden motives in others.",

    "J1": "I am not interested in abstract ideas.",
    "J2": "I am easily discouraged.",
    "J3": "I am not afraid of providing criticism.",
    "J4": "I disclose my intimate thoughts.",
    "J5": "I don't like action movies.",
    "J6": "I want everything to be 'just right.'",
    "J7": "I feel threatened easily.",
    "J8": "I am the last to laugh at a joke.",
    "J9": "I enjoy discussing movies and books with others.",
    "J10": "I joke around a lot.",

    "K1": "I have a poor vocabulary.",
    "K2": "I dislike loud music.",
    "K3": "I make insightful remarks.",
    "K4": "I enjoy bringing people together.",
    "K5": "I get chores done right away.",
    "K6": "I reflect on things before acting.",
    "K7": "I am not bothered by disorder.",
    "K8": "I don't like to get involved in other people's problems.",
    "K9": "I break rules.",
    "K10": "I can take strong measures.",

    "L1": "I love large parties.",
    "L2": "I do not like poetry.",
    "L3": "I believe that others have good intentions.",
    "L4": "I leave a mess in my room.",
    "L5": "I put off unpleasant tasks.",
    "L6": "I oppose authority.",
    "L7": "I resist authority.",
    "L8": "I readily overcome setbacks.",
    "L9": "I get confused easily.",
    "L10": "I know how to comfort others.",

    "M1": "I am open about myself to others.",
    "M2": "I rarely notice my emotional reactions.",
    "M3": "I amuse my friends.",
    "M4": "I love to think up new ways of doing things.",
    "M5": "I dislike works of fiction.",
    "M6": "I do not enjoy watching dance performances.",
    "M7": "I start conversations.",
    "M8": "I make friends easily.",
    "M9": "I often feel blue.",
    "M10": "I counter others' arguments.",

    "N1": "I am not interested in theoretical discussions.",
    "N2": "I seek quiet.",
    "N3": "I have frequent mood swings.",
    "N4": "I learn quickly.",
    "N5": "I rarely look for a deeper meaning in things.",
    "N6": "I like to read.",
    "N7": "I keep my thoughts to myself.",
    "N8": "I try to avoid complex people.",
    "N9": "I reveal little about myself.",
    "N10": "I am not bothered by messy people.",

    "O1": "I consider myself an average person.",
    "O2": "I like order.",
    "O3": "I avoid philosophical discussions.",
    "O4": "I am annoyed by others' mistakes.",
    "O5": "I cry during movies.",
    "O6": "I am not really interested in others.",
    "O7": "I believe that people are essentially evil.",
    "O8": "I know how to get around the rules.",
    "O9": "I seldom joke around.",
    "O10": "I carry the conversation to a higher level.",

    "P1": "I spend time thinking about past mistakes.",
    "P2": "I talk to a lot of different people at parties.",
    "P3": "I bottle up my feelings.",
    "P4": "I want to be left alone.",
    "P5": "I take an interest in other people's lives.",
    "P6": "I am wary of others.",
    "P7": "I enjoy teamwork.",
    "P8": "I have little to say.",
    "P9": "I believe laws should be strictly enforced.",
    "P10": "I do things by the book.",
}


In [4]:
likert_map = {
    "1": "Strongly Disagree",
    "2": "Disagree",
    "3": "Neutral",
    "4": "Agree",
    "5": "Strongly Agree"
}


In [5]:
df.columns

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       ...
       'P7', 'P8', 'P9', 'P10', 'age', 'gender', 'accuracy', 'country',
       'source', 'elapsed'],
      dtype='object', length=169)

In [6]:
# Create a new column with all responses joined
df["full_text"] = df.apply(lambda row: " ".join(row.values.astype(str)), axis=1)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P8,P9,P10,age,gender,accuracy,country,source,elapsed,full_text
0,1,4,2,3,3,2,3,4,4,3,...,2,5,2,17,1,92,US,6,914,1 4 2 3 3 2 3 4 4 3 4 4 5 4 5 4 5 4 1 2 1 1 1 ...
1,4,3,4,3,4,4,4,4,2,2,...,2,2,2,37,1,100,US,1,891,4 3 4 3 4 4 4 4 2 2 4 4 4 4 5 4 3 2 3 2 4 1 1 ...
2,3,4,4,4,4,4,4,3,2,2,...,3,3,3,31,1,80,US,6,903,3 4 4 4 4 4 4 3 2 2 4 4 5 5 4 4 4 4 2 2 2 2 2 ...
3,4,5,4,4,4,3,3,2,2,2,...,3,4,4,32,1,93,US,1,806,4 5 4 4 4 3 3 2 2 2 4 2 4 5 4 5 4 4 3 3 3 2 4 ...
4,4,0,4,4,4,3,5,1,2,4,...,2,3,2,46,2,87,NZ,1,1826,4 0 4 4 4 3 5 1 2 4 2 4 4 5 5 4 4 5 4 1 5 1 2 ...


In [7]:
# Combine question and answer into one sentence per row
def combine_q_a(row):
    combined = []
    for col in row.index:
        if col in question_map:
            q = question_map[col]
            raw_answer = str(row[col])
            response = likert_map.get(raw_answer, raw_answer)  # fallback in case of bad data
            combined.append(f"{q} Answer: {response}.")
    return " ".join(combined)

# Apply to each row of the DataFrame
df["full_text"] = df.apply(combine_q_a, axis=1)
df = df[["full_text", "accuracy"]]
df

,full_text,accuracy
0,I take time out for others. Answer: Strongly D...,92
1,I take time out for others. Answer: Agree. I k...,100
2,I take time out for others. Answer: Neutral. I...,80
3,I take time out for others. Answer: Agree. I k...,93
4,I take time out for others. Answer: Agree. I k...,87
...,...,...
49154,I take time out for others. Answer: Strongly D...,100
49155,I take time out for others. Answer: Agree. I k...,50
49156,I take time out for others. Answer: Disagree. ...,85
49157,I take time out for others. Answer: Agree. I k...,95


In [8]:
df.to_csv("df.csv", index=True)

In [9]:
# Load a pretrained NLP model that understands sentence meanings
model = SentenceTransformer('all-MiniLM-L6-v2')
trait_descriptions = {
    "Warmth": "Friendly, compassionate, empathetic, emotionally supportive, affectionate, considerate, helpful to others.",
    "Reasoning": "Analytical, logical thinker, intelligent, enjoys complex problem-solving, quick learner, curious mind.",
    "Emotional Stability": "Calm under pressure, emotionally resilient, stable, not easily upset or anxious, steady personality.",
    "Dominance": "Assertive, takes charge, confident, competitive, authoritative, persuasive, prefers to lead.",
    "Liveliness": "Energetic, sociable, fun-loving, spontaneous, expressive, full of life and enthusiasm.",
    "Rule-Consciousness": "Responsible, conscientious, adheres to rules, moral, law-abiding, values tradition and order.",
    "Social Boldness": "Outgoing, fearless in social settings, talkative, brave, adventurous, not easily embarrassed.",
    "Sensitivity": "Emotionally aware, caring, aesthetic, gentle, moved by art and beauty, deeply affected by others' feelings.",
    "Vigilance": "Cautious, alert, skeptical, guarded, watchful, aware of possible deception or harm.",
    "Abstractedness": "Imaginative, dreamy, philosophical, creative thinker, lives in ideas, abstract over practical.",
    "Privateness": "Reserved, discreet, secretive, avoids sharing personal thoughts, emotionally distant, keeps to self.",
    "Apprehension": "Self-critical, anxious, worried, insecure, doubtful, often questions decisions and actions.",
    "Openness to Change": "Adaptable, curious, flexible, experimental, open-minded, welcomes novelty and change.",
    "Self-Reliance": "Independent, self-sufficient, prefers working alone, makes decisions without depending on others.",
    "Perfectionism": "Detail-oriented, organized, disciplined, high personal standards, strives for precision and flawlessness.",
    "Tension": "Impatient, stressed, easily frustrated, restless, under pressure, tends to worry or overthink."
}

# Encode trait descriptions
trait_vectors = model.encode(list(trait_descriptions.values()), show_progress_bar=True)

# Map trait names to their embedding
trait_vector_dict = dict(zip(trait_descriptions.keys(), trait_vectors))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**HERE WE USE SAMPLE DATA BUT WHEN TRAINING OUR MODEL WE'RE GONNA USE ALL THE DATA**

In [10]:
# Sample participant embeddings from your earlier step
sample_df = df.sample(300, random_state=42)
participant_texts = sample_df["full_text"].tolist()
participant_embeddings = model.encode(participant_texts, show_progress_bar=True)

# Calculate cosine similarity between each participant and each trait
similarity_matrix = []
for person_vec in participant_embeddings:
    person_scores = {
        trait: float(cosine_similarity([person_vec], [trait_vec])[0][0])
        for trait, trait_vec in trait_vector_dict.items()
    }
    similarity_matrix.append(person_scores)

# Create a DataFrame with trait scores
trait_scores_df = pd.DataFrame(similarity_matrix)
trait_scores_df.head(20)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

,Warmth,Reasoning,Emotional Stability,Dominance,Liveliness,Rule-Consciousness,Social Boldness,Sensitivity,Vigilance,Abstractedness,Privateness,Apprehension,Openness to Change,Self-Reliance,Perfectionism,Tension
0,0.368059,0.298694,0.398410,0.323615,0.274359,0.268645,0.292844,0.389999,0.349733,0.284752,0.378729,0.450125,0.310975,0.380782,0.168506,0.344553
1,0.359427,0.291089,0.382453,0.303740,0.266221,0.250808,0.284371,0.384194,0.342133,0.281209,0.370056,0.434253,0.307712,0.374285,0.164574,0.336499
2,0.345469,0.299468,0.335209,0.272128,0.259977,0.231233,0.266696,0.405663,0.291862,0.289705,0.342201,0.390346,0.306104,0.344445,0.154595,0.287029
3,0.356640,0.310935,0.354193,0.286064,0.265884,0.246558,0.272608,0.398806,0.309747,0.296986,0.357043,0.412435,0.304717,0.362210,0.160245,0.306918
4,0.342322,0.285851,0.360223,0.291122,0.248398,0.242378,0.270034,0.366702,0.313234,0.264199,0.359733,0.416082,0.290517,0.373875,0.156882,0.316446
5,0.345865,0.296937,0.335169,0.288520,0.259365,0.244323,0.263518,0.399862,0.302713,0.294341,0.354028,0.404575,0.305010,0.359692,0.162869,0.291396
6,0.337733,0.289860,0.329633,0.278801,0.249463,0.229271,0.267778,0.384316,0.285464,0.281797,0.360660,0.405789,0.291199,0.355164,0.157452,0.297700
7,0.333172,0.273168,0.345811,0.300266,0.223862,0.247218,0.265070,0.353587,0.313235,0.254729,0.345518,0.405078,0.269210,0.357418,0.170680,0.307689
8,0.356040,0.310986,0.360663,0.305693,0.275932,0.251567,0.277030,0.411273,0.325598,0.301197,0.374086,0.423776,0.322799,0.377185,0.143947,0.319243
9,0.366548,0.291544,0.374680,0.321204,0.264170,0.267861,0.289164,0.381147,0.348027,0.277966,0.373504,0.440097,0.299462,0.379191,0.178325,0.327162


In [11]:
# Normalize scores from 0 to 100
normalized_df = trait_scores_df.apply(lambda x: 100 * (x - x.min()) / (x.max() - x.min()))
final_df_normalized = pd.concat([sample_df.reset_index(drop=True), normalized_df.reset_index(drop=True)], axis=1)
final_df_normalized.head()

,full_text,accuracy,Warmth,Reasoning,Emotional Stability,Dominance,Liveliness,Rule-Consciousness,Social Boldness,Sensitivity,Vigilance,Abstractedness,Privateness,Apprehension,Openness to Change,Self-Reliance,Perfectionism,Tension
0,I take time out for others. Answer: Strongly A...,85,72.696946,55.879647,86.595239,74.730329,49.738209,85.772014,49.149014,66.110134,68.475642,44.910118,58.945643,72.088379,51.915266,77.917893,62.968952,76.904691
1,I take time out for others. Answer: Strongly A...,90,63.456878,49.548924,73.095225,55.969639,44.460194,61.540919,42.135388,60.901366,62.887832,42.568549,49.171073,58.048049,49.494479,68.335451,58.410912,68.580027
2,I take time out for others. Answer: Agree. I k...,60,48.513381,56.524244,33.127517,26.129827,40.409677,34.946974,27.503395,80.163714,25.924803,48.184228,17.774120,19.208669,48.301863,24.327098,46.841851,17.444779
3,I take time out for others. Answer: Agree. I k...,75,60.472856,66.070384,49.188190,39.284143,44.241370,55.767261,32.397412,74.011483,39.075030,52.996612,34.503356,38.748138,47.272823,50.526546,53.392124,38.003098
4,I take time out for others. Answer: Agree. I k...,90,45.144399,45.188139,54.288885,44.058408,32.899507,50.087694,30.266405,45.208404,41.638763,31.324406,37.535019,41.974254,36.737413,67.731788,49.492997,47.852033


In [12]:
trait_columns = [
    "Warmth", "Reasoning", "Emotional Stability", "Dominance", "Liveliness",
    "Rule-Consciousness", "Social Boldness", "Sensitivity", "Vigilance",
    "Abstractedness", "Privateness", "Apprehension", "Openness to Change",
    "Self-Reliance", "Perfectionism", "Tension"
]

high_accuracy_df = final_df_normalized[final_df_normalized["accuracy"] > 80].reset_index(drop=True)
features_df = high_accuracy_df[trait_columns]
features_df.head()

,Warmth,Reasoning,Emotional Stability,Dominance,Liveliness,Rule-Consciousness,Social Boldness,Sensitivity,Vigilance,Abstractedness,Privateness,Apprehension,Openness to Change,Self-Reliance,Perfectionism,Tension
0,72.696946,55.879647,86.595239,74.730329,49.738209,85.772014,49.149014,66.110134,68.475642,44.910118,58.945643,72.088379,51.915266,77.917893,62.968952,76.904691
1,63.456878,49.548924,73.095225,55.969639,44.460194,61.540919,42.135388,60.901366,62.887832,42.568549,49.171073,58.048049,49.494479,68.335451,58.410912,68.580027
2,45.144399,45.188139,54.288885,44.058408,32.899507,50.087694,30.266405,45.208404,41.638763,31.324406,37.535019,41.974254,36.737413,67.731788,49.492997,47.852033
3,40.231913,48.525399,28.410502,32.428438,33.590039,32.281487,28.399184,61.011341,21.220846,42.956585,38.579903,32.869296,37.243110,40.134912,50.153361,28.475431
4,35.349545,34.629986,42.097137,52.690435,16.984494,56.663800,26.157404,33.441624,41.639968,25.065015,21.512976,32.240571,20.929223,43.459544,65.488823,38.800305


In [ ]:
#embeddings_df = pd.DataFrame(embeddings)
#embeddings_df.head()
#print(embeddings[0])  # Shows the embedding vector of the first participant


In [ ]:
#we are working with similarity based analysis since we don't have any alabeled datasets about correlations between traits and individuals

MACHINE LEARNING MODEL + MODEL TRAINING

In [14]:
#sample_df = df.sample(300, random_state=42)
#participant_texts = sample_df["full_text"].tolist()
#HERE WE USE SAMPLE DATA BUT WHEN TRAINING OUR MODEL WE'RE GONNA USE ALL THE DATA
participant_texts = df["full_text"].tolist()
participant_embeddings = model.encode(participant_texts, show_progress_bar=True)

# Calculate cosine similarity between each participant and each trait
similarity_matrix = []
for person_vec in participant_embeddings:
    person_scores = {
        trait: float(cosine_similarity([person_vec], [trait_vec])[0][0])
        for trait, trait_vec in trait_vector_dict.items()
    }
    similarity_matrix.append(person_scores)

# Create a DataFrame with trait scores
trait_scores_df = pd.DataFrame(similarity_matrix)
#trait_scores_df.head(20)
# Normalize scores from 0 to 100
normalized_df = trait_scores_df.apply(lambda x: 100 * (x - x.min()) / (x.max() - x.min()))
final_df_normalized = pd.concat([df.reset_index(drop=True), normalized_df.reset_index(drop=True)], axis=1)
#final_df_normalized.head()
trait_columns = [
    "Warmth", "Reasoning", "Emotional Stability", "Dominance", "Liveliness",
    "Rule-Consciousness", "Social Boldness", "Sensitivity", "Vigilance",
    "Abstractedness", "Privateness", "Apprehension", "Openness to Change",
    "Self-Reliance", "Perfectionism", "Tension"
]

high_accuracy_df = final_df_normalized[final_df_normalized["accuracy"] > 80].reset_index(drop=True)
features_df = high_accuracy_df[trait_columns]
features_df.head()

Batches:   0%|          | 0/1537 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
test_data = {
    "Name": ["Badreddine", "Ali", "Yasser"],
    "Warmth": [0.3, 1.2, 1.0],
    "Reasoning": [0.3, 1.0, 0.9],
    "Emotional_Stability": [2, 1.7, 1.5],
    "Dominance": [0.4, 1.5, 1.0],
    "Liveliness": [1.4, 0.9, 1.6],
    "Rule_Consciousness": [0, 1.8, 2],
    "Social_Boldness": [2, 1.5, 1.2],
    "Sensitivity": [1.4, 1.6, 1.5],
    "Vigilance": [1.4, 0.9, 1.1],
    "Abstractedness": [0.8, 1.2, 1.3],
    "Privateness": [2, 0.2, 0.7],
    "Apprehension": [0.8, 1.0, 0.9],
    "Openness_to_Change": [2, 1.5, 1.6],
    "Self_Reliance": [2, 1.0, 1.1],
    "Perfectionism": [2, 1.8, 1.5],
    "Tension": [2, 1.1, 1.2],
}

#df = pd.DataFrame(data)
#df.set_index("Name", inplace=True)
#df.head()

#Normalize it from 0–2 to 1–100 scale.
#normalized_df = trait_scores_df.apply(lambda x: 100 * (x - x.min()) / (x.max() - x.min()))
#normalized_test_data = data.apply(lambda x: 100 * (x - x.min()) / (x.max() - x.min()))

test_df = pd.DataFrame(test_data)

# Drop the "Name" column to only keep the trait scores
test_features = test_df.drop("Name", axis=1)

# Compute cosine similarity between all members
similarity_matrix = cosine_similarity(test_features)

# Convert to readable format
similarity_df = pd.DataFrame(similarity_matrix, index=test_df["Name"], columns=test_df["Name"])
print("Similarity matrix between team members:")
print(similarity_df.round(2))

Similarity matrix between team members:
Name        Badreddine   Ali  Yasser
Name                                
Badreddine        1.00  0.81    0.85
Ali               0.81  1.00    0.97
Yasser            0.85  0.97    1.00
